In [1]:
import os
import openai 

openai.api_type="azure"
openai.api_version="2023-05-15"

OPENAI_API_BASE = "XXX"
OPENAI_API_KEY = "XXX"

openai.api_base = OPENAI_API_BASE
openai.api_key = OPENAI_API_KEY


In [2]:
models = openai.Model.list()
print(models)

{
  "data": [
    {
      "capabilities": {
        "chat_completion": false,
        "completion": true,
        "embeddings": false,
        "fine_tune": false,
        "inference": false
      },
      "created_at": 1646092800,
      "deprecation": {
        "fine_tune": 1709251200,
        "inference": 1709251200
      },
      "id": "ada",
      "lifecycle_status": "preview",
      "object": "model",
      "status": "succeeded",
      "updated_at": 1646092800
    },
    {
      "capabilities": {
        "chat_completion": false,
        "completion": true,
        "embeddings": false,
        "fine_tune": false,
        "inference": false
      },
      "created_at": 1646092800,
      "deprecation": {
        "fine_tune": 1709251200,
        "inference": 1709251200
      },
      "id": "babbage",
      "lifecycle_status": "preview",
      "object": "model",
      "status": "succeeded",
      "updated_at": 1646092800
    },
    {
      "capabilities": {
        "chat_completion": f

In [3]:
import pandas as pd 
df = pd.read_csv("./sales_data_sample.csv")
df.head()             

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME
0,10107,30,95.70,2871.00,2/24/2003 0:00,1,2,2003,Motorcycles,2125557818,897 Long Airport Avenue,NYC,NY,10022,USA,Yu,Kwai
1,10121,34,81.35,2765.90,5/7/2003 0:00,2,5,2003,Motorcycles,26.47.1555,59 rue de l'Abbaye,Reims,NaN,51100,France,Henriot,Paul
2,10134,41,94.74,3884.34,7/1/2003 0:00,3,7,2003,Motorcycles,+33 1 46 62 7555,27 rue du Colonel Pierre Avia,Paris,NaN,75508,France,Da Cunha,Daniel
3,10145,45,83.26,3746.70,8/25/2003 0:00,3,8,2003,Motorcycles,6265557265,78934 Hillside Dr.,Pasadena,CA,90003,USA,Young,Julie
4,10159,49,100.00,5205.27,10/10/2003 0:00,4,10,2003,Motorcycles,6505551386,7734 Strong St.,San Francisco,CA,NaN,USA,Brown,Julie


In [4]:
df.groupby('QTR_ID').sum()['SALES']

QTR_ID
1    2350817.73
2    2048120.30
3    1758910.81
4    3874780.01
Name: SALES, dtype: float64

In [5]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import text

In [6]:
temp_db = create_engine('sqlite:///:memory:', echo=True)
data = df.to_sql(name = "Sales", con=temp_db)
 

2023-06-06 22:40:07,100 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-06 22:40:07,102 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Sales")
2023-06-06 22:40:07,103 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-06 22:40:07,103 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Sales")
2023-06-06 22:40:07,104 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-06-06 22:40:07,105 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Sales" (
	"index" BIGINT, 
	"ORDERNUMBER" BIGINT, 
	"QUANTITYORDERED" BIGINT, 
	"PRICEEACH" FLOAT, 
	"SALES" FLOAT, 
	"ORDERDATE" TEXT, 
	"QTR_ID" BIGINT, 
	"MONTH_ID" BIGINT, 
	"YEAR_ID" BIGINT, 
	"PRODUCTLINE" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT
)


2023-06-06 22:40:07,105 INFO sqlalchemy.engine.Engine [no key 0.00028s] ()
2023-06-06 22:40:07,106 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_Sales_index" ON "Sales" (

In [7]:
with temp_db.connect() as conn:
    result = conn.execute(text("SELECT SUM(SALES) FROM Sales"))
    

2023-06-06 22:40:12,658 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-06 22:40:12,658 INFO sqlalchemy.engine.Engine SELECT SUM(SALES) FROM Sales
2023-06-06 22:40:12,659 INFO sqlalchemy.engine.Engine [generated in 0.00131s] ()
2023-06-06 22:40:12,660 INFO sqlalchemy.engine.Engine ROLLBACK


In [8]:
result.all()

[(10032628.85000001,)]

In [9]:
def create_table_definition(df):
    prompt = """### sqlite SQL table, with it properties:
    #
    # Sales({})
    #
    """.format(",".join(str(col) for col in df.columns))
    return prompt

In [10]:
df.columns

Index(['ORDERNUMBER', 'QUANTITYORDERED', 'PRICEEACH', 'SALES', 'ORDERDATE',
       'QTR_ID', 'MONTH_ID', 'YEAR_ID', 'PRODUCTLINE', 'PHONE', 'ADDRESSLINE1',
       'CITY', 'STATE', 'POSTALCODE', 'COUNTRY', 'CONTACTLASTNAME',
       'CONTACTFIRSTNAME'],
      dtype='object')

In [11]:
[str(col) for col in df.columns]

['ORDERNUMBER',
 'QUANTITYORDERED',
 'PRICEEACH',
 'SALES',
 'ORDERDATE',
 'QTR_ID',
 'MONTH_ID',
 'YEAR_ID',
 'PRODUCTLINE',
 'PHONE',
 'ADDRESSLINE1',
 'CITY',
 'STATE',
 'POSTALCODE',
 'COUNTRY',
 'CONTACTLASTNAME',
 'CONTACTFIRSTNAME']

In [12]:
",".join(str(col) for col in df.columns)


'ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME'

In [13]:
print(create_table_definition(df))

### sqlite SQL table, with it properties:
    #
    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME)
    #
    


In [14]:
def prompt_input():
    nlp_text = input("Enter the info you want to query: ")
    return nlp_text



In [15]:
prompt_input()


'§sss'

In [17]:
def combine_prompts(df, query_prompt):
    definition = create_table_definition(df)
    query_init_string = f"### A Query to answer: {query_prompt}\nSELECT"
    return definition + query_init_string

In [21]:
nlp_text = prompt_input()
prompt = combine_prompts(df, nlp_text)
print(prompt)


### sqlite SQL table, with it properties:
    #
    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME)
    #
    ### A Query to answer: sales per quarter
SELECT


In [34]:
from langchain.llms import AzureOpenAI 

llm = AzureOpenAI(
    deployment_name="gpt-35-turbo",
    temperature=0,
    max_tokens=150,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop = ['#',';']
) 
response = "SELECT" + llm(prompt)


WARNING! stop is not default parameter.
                    stop was transferred to model_kwargs.
                    Please confirm that stop is what you intended.


In [35]:
print(response)

SELECT QTR_ID, SUM(SALES) FROM Sales GROUP BY QTR_ID


In [36]:
with temp_db.connect() as conn:
    result = conn.execute(text(response))


2023-06-06 23:08:40,882 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-06 23:08:40,884 INFO sqlalchemy.engine.Engine SELECT QTR_ID, SUM(SALES) FROM Sales GROUP BY QTR_ID
2023-06-06 23:08:40,884 INFO sqlalchemy.engine.Engine [generated in 0.00216s] ()
2023-06-06 23:08:40,888 INFO sqlalchemy.engine.Engine ROLLBACK


In [37]:
print(result.all())

[(1, 2350817.7300000004), (2, 2048120.2999999986), (3, 1758910.8099999994), (4, 3874780.01)]


In [ ]:
def handle_response(response):
    query = response['choices'][0]['text']
    if query.startswith(" "):
        query= "SELECT" + query
    return query